## 第6章 数据加载、存储与文件格式

In [1]:
import pandas as pd
import numpy as np
np.random.seed(2019)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## 读写文本格式的数据

In [2]:
df = pd.read_csv('./examples/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [3]:
pd.read_table('./examples/ex1.csv', sep = '.')    # sep，指定分隔符

/home/ubuntu/anaconda3/envs/tf/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.


,"a,b,c,d,message"
0,"1,2,3,4,hello"
1,"5,6,7,8,world"
2,"9,10,11,12,foo"


In [4]:
!cat ./examples/ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [5]:
pd.read_csv('./examples/ex2.csv', header=None) # pandas分配默认的列名

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [6]:
pd.read_csv('./examples/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])  # 自定义列名

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [7]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('./examples/ex2.csv', names=names, index_col='message')  # index_col指定索引

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [8]:
parsed = pd.read_csv('./examples/csv_mindex.csv', index_col=['key1', 'key2']) # 层次化索引
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [9]:
!cat ./examples/ex3.txt

            A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491


In [10]:
result = pd.read_table('./examples/ex3.txt', sep='\s+') # 传入正则表达式表示分隔符
result

/home/ubuntu/anaconda3/envs/tf/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.


,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [11]:
# 使用skiprows跳过文件的第一行、第三行和第四行
!cat ./examples/ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [12]:
pd.read_csv('./examples/ex4.csv', skiprows=[0,2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [13]:
# 缺失值处理
!cat ./examples/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

In [14]:
result = pd.read_csv('./examples/ex5.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [15]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [16]:
# na_values 可以用一个列表或集合的字符串表示缺失值
result = pd.read_csv('./examples/ex5.csv', na_values='NULL')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [17]:
# 字典的各列可以使用不同的NA标记值
sentinels = {'message':['foo', 'NA'], 'something':['two']}
pd.read_csv('./examples/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


### 逐块读取文本文件

In [18]:
# 设置pandas显示的更紧一些
pd.options.display.max_rows = 10

In [19]:
result = pd.read_csv('./examples/ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [20]:
# 如果只想读取几行，通过nrows进行指定：
pd.read_csv('./examples/ex6.csv', nrows=8)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
5,1.817480,0.742273,0.419395,-2.251035,Q
6,-0.776764,0.935518,-0.332872,-1.875641,U
7,-0.913135,1.530624,-0.572657,0.477252,K


In [21]:
# 要逐块读取文件，可以指定chunksize(行数)
chunker = pd.read_csv('./examples/ex6.csv', chunksize=1000)
chunker

In [22]:
# read_csv返回的这个TextFileReader对象使你可以根据chunksize对文件进行逐块迭代
# 比如说我们可迭代处理ex6.csv，将值技术聚合到'key'列中：
chunker = pd.read_csv('./examples/ex6.csv', chunksize=1000)
tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)
tot = tot.sort_values(ascending=False)
tot

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
     ...  
5    157.0
2    152.0
0    151.0
9    150.0
1    146.0
Length: 36, dtype: float64

### 将数据写出到文本格式

In [23]:
data = pd.read_csv('./examples/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [24]:
# 利用to_csv方法，将数据写到一个以逗号分割的文件
data.to_csv('./examples/out.csv')
!cat ./examples/out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [25]:
# 还可以使用其他分隔符,sep指定
import sys
data.to_csv(sys.stdout, sep='*')

*something*a*b*c*d*message
0*one*1*2*3.0*4*
1*two*5*6**8*world
2*three*9*10*11.0*12*foo


In [26]:
# 缺失值在输出结果里会被表示为空字符串，可以na_rep表示为自定义的标记值
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [27]:
# 如果没有设置其他选项，则会写出行和列的标签，当然，然们也都可以被禁用
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [28]:
# 只写出一部分列，并以指定的顺序排列
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


In [29]:
# Series也有一个to_csv方法：
dates = pd.date_range('1/1/1990', periods=10)
ts = pd.Series(np.arange(10), index=dates)
ts.to_csv('./examples/tseries.csv')
!cat ./examples/tseries.csv

1990-01-01,0
1990-01-02,1
1990-01-03,2
1990-01-04,3
1990-01-05,4
1990-01-06,5
1990-01-07,6
1990-01-08,7
1990-01-09,8
1990-01-10,9


/home/ubuntu/anaconda3/envs/tf/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  after removing the cwd from sys.path.


### 处理分隔符格式

有一些数据需要进行特殊处理

In [30]:
!cat ./examples/ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


In [31]:
# 对于任何单字符分隔符文件，可以直接使用Python内置的csv模块。
# 将任意已打开的文件或文件型的对象传给csv.reader

import csv

f = open('./examples/ex7.csv')

reader = csv.reader(f)

In [32]:
# 对reader进行迭代将会为每行产生一个元祖（并溢出了所有的引号）
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


为了使数据格式合乎要求，需要对其整理

In [33]:
# 读取文件到一个多行列表
with open('./examples/ex7.csv') as f:
    lines = list(csv.reader(f))

In [34]:
# 将这些行分为标题行和数据行
header, values = lines[0], lines[1:]

In [35]:
# 使用字典构造式和zip(*values)，后者将行转置为列，创建数据列的字典
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

## JSON数据
JSON（JavaScript Object Notation的简称）已经成为通过HTTP
请求在Web浏览器和其他应⽤程序之间发送数据的标准格式之
⼀。它是⼀种⽐表格型⽂本格式（如CSV）灵活得多的数据格
式。

In [36]:

obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

In [37]:
import json
result = json.loads(obj)
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

In [38]:
# json.dumps则将Python对象转换成JSON格式：
asjson = json.dumps(result)

In [39]:
# 将（⼀个或⼀组）JSON对象转换为DataFrame或其他便于分析的数据结构
# 最简单⽅便的⽅式是：向DataFrame构造器传⼊⼀个字典的列表（就是原先的JSON对象），并选取数据字段的⼦集
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings

,name,age
0,Scott,30
1,Katie,38


In [40]:
!cat ./examples/example.json

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


In [41]:
# pandas.read_json可以⾃动将特别格式的JSON数据集转换为Series或DataFrame
# pandas.read_json的默认选项假设JSON数组中的每个对象是表格中的⼀⾏

data = pd.read_json('./examples/example.json')
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [42]:
# 将数据从pandas输出到JSON，可以使⽤to_json
print(data.to_json())

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}


## XML和HTML： Web信息收集
Python有许多可以读写常⻅的HTML和XML格式数据的库，包括
lxml、 Beautiful Soup和html5lib。 lxml的速度⽐较快，但其它的
库处理有误的HTML或XML⽂件更好

pandas有⼀个内置的功能， read_html，它可以使⽤lxml和
Beautiful Soup⾃动将HTML⽂件中的表格解析为DataFrame对
象




In [43]:
tables = pd.read_html('./examples/fdic_failed_bank_list.html')
len(tables)

1

In [44]:
failures = tables[0]
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [45]:
# 计算按年份计算倒闭的银⾏数

close_timestamps = pd.to_datetime(failures['Closing Date'])
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, Length: 15, dtype: int64

### 利用lxml.objectify解析XML

XML（Extensible Markup Language）是另⼀种常⻅的⽀持分
层、嵌套数据以及元数据的结构化数据格式。本书所使⽤的这些
⽂件实际上来⾃于⼀个很⼤的XML⽂档。

pandas.read_html函数，它可以使⽤lxml或
Beautiful Soup从HTML解析数据。 XML和HTML的结构很相似，
但XML更为通⽤

In [47]:
# ⽤lxml.objectify解析该⽂件，然后通过getroot得到该XML⽂件的根节点的引⽤
from lxml import objectify

path = './datasets/mta_perf/Performance_MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot()

In [48]:
# root.INDICATOR返回⼀个⽤于产⽣各个<INDICATOR>XML元素的⽣成器。
# 对于每条记录，我们可以⽤标记名（如YTD_ACTUAL）和数据值填充⼀个字典（排除⼏个标记）

data = []

skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ',
               'DESIRED_CHANGE', 'DECIMAL_PLACES']

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [50]:
# 将词典转换为一个Dataframe
perf = pd.DataFrame(data)
perf.head()

,AGENCY_NAME,CATEGORY,DESCRIPTION,FREQUENCY,INDICATOR_NAME,INDICATOR_UNIT,MONTHLY_ACTUAL,MONTHLY_TARGET,PERIOD_MONTH,PERIOD_YEAR,YTD_ACTUAL,YTD_TARGET
0,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,1,2008,96.9,95
1,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95,95,2,2008,96,95
2,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,3,2008,96.3,95
3,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,98.3,95,4,2008,96.8,95
4,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95.8,95,5,2008,96.6,95


XML数据可以⽐本例复杂得多。每个标记都可以有元数据。看看
下⾯这个HTML的链接标签

In [51]:
from io import StringIO
tag = '<a href="http://www.google.com">Google</a>'
root = objectify.parse(StringIO(tag)).getroot()

现在就可以访问标签或链接⽂本中的任何字段了（如href）：

In [54]:
root

<Element a at 0x7f7189a4e688>

In [55]:
root.get('href')

'http://www.google.com'

## 二进制数据格式

实现数据的⾼效⼆进制格式存储最简单的办法之⼀是使⽤Python
内置的pickle序列化。 

pandas对象都有⼀个⽤于将数据以pickle格式保存到磁盘上的to_pickle⽅法：

In [57]:
frame = pd.read_csv('./examples/ex1.csv')

frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [58]:
frame.to_pickle('./examples/frame_pickle')

你可以通过pickle直接读取被pickle化的数据，或是使⽤更为⽅便
的pandas.read_pickle：

In [61]:
pd.read_pickle('./examples/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [62]:
!cat ./examples/frame_pickle

���      �pandas.core.frame��	DataFrame���)��}�(�_data��pandas.core.internals.managers��BlockManager���)��(]�(�pandas.core.indexes.base��
�pandas.core.indexes.range��
RangeIndex���}�(h+N�start�K �stop�K�step�Ku��R�e]�(hhK ��h��R�(KKK��h�i8�K K��R�(K�<�NNNJ����J����K t�b�C`              	                     
h}�(hhhK ��h��R�(KK��h!�]�h)at�bh+Nu��R�e}��0.14.1�}�(�axes�h
�blocks�]�(}�(�values�h:�mgr_locs��builtins��slice���K KK��R�u}�(hfhEhghjKKK��R�ueust�b�_typ��	dataframe��	_metadata�]�ub.

*注意： pickle仅建议⽤于短期存储格式。其原因是很难保证该
格式永远是稳定的；今天pickle的对象可能⽆法被后续版本的
库unpickle出来。虽然我尽⼒保证这种事情不会发⽣在
pandas中，但是今后的某个时候说不定还是得“打破”该pickle
格式。*

### 使用HDF5格式

HDF5是⼀种存储⼤规模科学数组数据的⾮常好的⽂件格式。它
可以被作为C库，带有许多语⾔的接⼝，如Java、 Python和
MATLAB等。 HDF5中的HDF指的是层次型数据格式
（hierarchical data format）。每个HDF5⽂件都含有⼀个⽂件系
统式的节点结构，它使你能够存储多个数据集并⽀持元数据。与
其他简单格式相⽐， HDF5⽀持多种压缩器的即时压缩，还能更
⾼效地存储重复模式数据。对于那些⾮常⼤的⽆法直接放⼊内存
的数据集， HDF5就是不错的选择，因为它可以⾼效地分块读
写。

虽然可以⽤PyTables或h5py库直接访问HDF5⽂件， pandas提供
了更为⾼级的接⼝，可以简化存储Series和DataFrame对象。
HDFStore类可以像字典⼀样，处理低级的细节：

In [93]:

frame = pd.DataFrame({'a': np.random.randn(100)})
store = pd.HDFStore('mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

In [94]:
# HDF5⽂件中的对象可以通过与字典⼀样的API进⾏获取：
store['obj1']

,a
0,0.579288
1,-0.491990
2,-0.279528
3,-2.031496
4,2.699457
...,...
95,-0.002640
96,0.244104
97,0.662426
98,0.836677


In [95]:
# HDFStore⽀持两种存储模式， 'fixed'和'table'。后者通常会更慢，但是⽀持使⽤特殊语法进⾏查询操作：
store.put('obj2', frame, format='table')
store.select('obj2', where=['index >= 10 and index <= 15'])
store.close()

In [97]:
frame.to_hdf('mydata2.h5', 'obj3', format='table')
pd.read_hdf('mydata2.h5', 'obj3', where=['index < 5'])

,a
0,0.579288
1,-0.491990
2,-0.279528
3,-2.031496
4,2.699457


In [99]:
os.remove('mydata.h5')

FileNotFoundError: [Errno 2] No such file or directory: 'mydata.h5'

注意： HDF5不是数据库。它最适合⽤作“⼀次写多次读”的数
据集。虽然数据可以在任何时候被添加到⽂件中，但如果同
时发⽣多个写操作，⽂件就可能会被破坏。

## 读取Excel文件

In [101]:
# 要使⽤ExcelFile，通过传递xls或xlsx路径创建⼀个实例：
xlsx = pd.ExcelFile('./examples/ex1.xlsx')

In [102]:
# 存储在表单中的数据可以read_excel读取到DataFrame
pd.read_excel(xlsx, 'Sheet1')

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [103]:
# 如果要读取⼀个⽂件中的多个表单，创建ExcelFile会更快，但你也可以将⽂件名传递到pandas.read_excel：
frame = pd.read_excel('./examples/ex1.xlsx', 'Sheet1')
frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [107]:
# 如果要将pandas数据写⼊为Excel格式，你必须⾸先创建⼀个ExcelWriter，
# 然后使⽤pandas对象的to_excel⽅法将数据写⼊到其中：

writer = pd.ExcelWriter('./examples/ex2.xlsx')
frame.to_excel(writer, 'Sheet1')
writer.save()

In [108]:
# 你还可以不使⽤ExcelWriter，⽽是传递⽂件的路径到to_excel：
frame.to_excel('./examples/ex2.xlsx')

## Web APIs交互


In [123]:
import requests
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
resp

<Response [200]>

In [124]:
data = resp.json()
data[0]['title']

'Infer names in MultiIndex.from_product'

In [125]:
issues = pd.DataFrame(data, columns=['number', 'title',
                                     'labels', 'state'])
issues

,number,title,labels,state
0,27895,Infer names in MultiIndex.from_product,[],open
1,27894,remove index column in SeriesGroupBy.nlargest,[],open
2,27893,Doc Fixes,[],open
3,27892,DataFrameGroupBy.quantile raises for non-numer...,"[{'id': 233160, 'node_id': 'MDU6TGFiZWwyMzMxNj...",open
4,27891,CLN: remove unnecessary validate_for_numeric_u...,[],open
...,...,...,...,...
25,27867,CI: Unpin 3.7.3 pins,"[{'id': 48070600, 'node_id': 'MDU6TGFiZWw0ODA3...",open
26,27865,Cannot use .ix in IntervaIndex('pandas._libs.i...,"[{'id': 2822098, 'node_id': 'MDU6TGFiZWwyODIyM...",open
27,27862,DOC: See also for DataFrame.iterrows() should ...,"[{'id': 134699, 'node_id': 'MDU6TGFiZWwxMzQ2OT...",open
28,27861,"DOC: Standardize use of ""Iterate"" and ""Iterato...","[{'id': 134699, 'node_id': 'MDU6TGFiZWwxMzQ2OT...",open


## 数据库交互

In [117]:
import sqlite3

query = """
CREATE TABLE test
(
a VARCHAR(20), b VARCHAR(20),
c REAL,        d INTEGER
);
"""

con = sqlite3.connect('mydata2.sqlite')
con.execute(query)
con.commit()

OperationalError: table test already exists

In [118]:
# 插入数据

data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"
con.executemany(stmt, data)
con.commit()

In [119]:
# 从表中选取数据时，⼤部分Python SQL驱动器（PyODBC、psycopg2、 MySQLdb、 pymssql等）都会返回⼀个元组列表：
cusor = con.execute('select * from test')
rows = cusor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5),
 ('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5),
 ('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [120]:
# 你可以将这个元组列表传给DataFrame构造器，但还需要列名（位于光标的description属性中）：
cusor.description
pd.DataFrame(rows, columns=[x[0] for x in cusor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
3,Atlanta,Georgia,1.25,6
4,Tallahassee,Florida,2.60,3
5,Sacramento,California,1.70,5
6,Atlanta,Georgia,1.25,6
7,Tallahassee,Florida,2.60,3
8,Sacramento,California,1.70,5


pandas有⼀个read_sql函数，
可以让你轻松的从SQLAlchemy连接读取数据。这⾥，我们⽤
SQLAlchemy连接SQLite数据库，并从之前创建的表读取数据

In [122]:
import sqlalchemy as sqla
db = sqla.create_engine('sqlite:///mydata2.sqlite')
pd.read_sql('select * from test', db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
3,Atlanta,Georgia,1.25,6
4,Tallahassee,Florida,2.60,3
5,Sacramento,California,1.70,5
6,Atlanta,Georgia,1.25,6
7,Tallahassee,Florida,2.60,3
8,Sacramento,California,1.70,5
